In [1]:
import cassio
from dotenv import load_dotenv
import os


load_dotenv()


os.environ["ASTRA_DB_APPLICATION_TOKEN"] = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
os.environ["ASTRA_DB_ID"] = os.getenv("ASTRA_DB_ID")

cassio.init(
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    database_id=os.environ["ASTRA_DB_ID"]
)


In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from acts_pdfs import get_acts

act_urls=get_acts()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import os
import pickle
import time
from langchain_community.document_loaders import UnstructuredURLLoader
from concurrent.futures import ThreadPoolExecutor, as_completed

# ✅ Step 1: Resume if partial_docs.pkl exists
if os.path.exists("partial_docs.pkl"):
    with open("partial_docs.pkl", "rb") as f:
        all_docs = pickle.load(f)
    print(f"🔄 Resumed from saved file with {len(all_docs)} docs.")
else:
    all_docs = []

# ✅ Load previously failed URLs (if any)
if os.path.exists("failed_urls.pkl"):
    with open("failed_urls.pkl", "rb") as f:
        previously_failed = pickle.load(f)
else:
    previously_failed = []

# 🔁 Optional: start fresh or continue from last
processed_urls = set(doc.metadata.get("source") for doc in all_docs if hasattr(doc, "metadata"))
act_urls_to_process = [url for url in act_urls if url not in processed_urls]

print(f"🚀 Starting fresh on {len(act_urls_to_process)} new URLs")

failed_urls = []

# ✅ Step 2: Load with retries + incremental saving
def fetch_doc(url):
    try:
        loader = UnstructuredURLLoader(urls=[url])
        return loader.load()
    except Exception as e:
        print(f"❌ Failed: {url} — {e}")
        failed_urls.append(url)
        return []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(fetch_doc, url): url for url in act_urls_to_process}
    for idx, future in enumerate(as_completed(futures), 1):
        docs = future.result()
        all_docs.extend(docs)

        # ✅ Save every 10 steps
        if idx % 10 == 0 or idx == len(act_urls_to_process):
            with open("partial_docs.pkl", "wb") as f:
                pickle.dump(all_docs, f)
            print(f"💾 Saved {len(all_docs)} docs so far at step {idx}.")

        print(f"✅ [{idx}/{len(act_urls_to_process)}] Loaded {len(docs)} docs")

# ✅ Final Save
with open("partial_docs.pkl", "wb") as f:
    pickle.dump(all_docs, f)

with open("failed_urls.pkl", "wb") as f:
    pickle.dump(failed_urls, f)

print(f"\n🎯 Total loaded: {len(all_docs)}")
print(f"❌ Failed URLs: {len(failed_urls)} saved to failed_urls.pkl")


🚀 Starting fresh on 871 new URLs


/home/bunnys-weapon/Documents/Langchain projects/dharmbantu/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ [1/871] Loaded 1 docs
✅ [2/871] Loaded 1 docs
✅ [3/871] Loaded 1 docs
✅ [4/871] Loaded 1 docs
✅ [5/871] Loaded 1 docs
✅ [6/871] Loaded 1 docs
✅ [7/871] Loaded 1 docs
✅ [8/871] Loaded 1 docs
✅ [9/871] Loaded 1 docs
💾 Saved 10 docs so far at step 10.
✅ [10/871] Loaded 1 docs
✅ [11/871] Loaded 1 docs
✅ [12/871] Loaded 1 docs
✅ [13/871] Loaded 1 docs
✅ [14/871] Loaded 1 docs
✅ [15/871] Loaded 1 docs
✅ [16/871] Loaded 1 docs
✅ [17/871] Loaded 1 docs
✅ [18/871] Loaded 1 docs
✅ [19/871] Loaded 1 docs
💾 Saved 20 docs so far at step 20.
✅ [20/871] Loaded 1 docs
✅ [21/871] Loaded 1 docs
✅ [22/871] Loaded 1 docs
✅ [23/871] Loaded 1 docs
✅ [24/871] Loaded 1 docs
✅ [25/871] Loaded 1 docs
✅ [26/871] Loaded 1 docs
✅ [27/871] Loaded 1 docs
✅ [28/871] Loaded 1 docs
✅ [29/871] Loaded 1 docs
💾 Saved 30 docs so far at step 30.
✅ [30/871] Loaded 1 docs
✅ [31/871] Loaded 1 docs
✅ [32/871] Loaded 1 docs
✅ [33/871] Loaded 1 docs
✅ [34/871] Loaded 1 docs
✅ [35/871] Loaded 1 docs
✅ [36/871] Loaded 1 docs
✅ [3

Error fetching or processing https://www.indiacode.nic.in/bitstream/123456789/2003/1/A2001-52.pdf, exception: HTTPSConnectionPool(host='www.indiacode.nic.in', port=443): Max retries exceeded with url: /bitstream/123456789/2003/1/A2001-52.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7cef245a80>: Failed to resolve 'www.indiacode.nic.in' ([Errno -3] Temporary failure in name resolution)"))
Error fetching or processing https://www.indiacode.nic.in/bitstream/123456789/2078/3/A2007-07.pdf, exception: HTTPSConnectionPool(host='www.indiacode.nic.in', port=443): Max retries exceeded with url: /bitstream/123456789/2078/3/A2007-07.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7cee9c9180>: Failed to resolve 'www.indiacode.nic.in' ([Errno -3] Temporary failure in name resolution)"))


✅ [264/871] Loaded 1 docs
✅ [265/871] Loaded 0 docs


...

✅ [455/871] Loaded 0 docs
✅ [456/871] Loaded 1 docs
✅ [457/871] Loaded 1 docs
✅ [458/871] Loaded 1 docs
✅ [459/871] Loaded 1 docs
💾 Saved 444 docs so far at step 460.
✅ [460/871] Loaded 1 docs
✅ [461/871] Loaded 1 docs
✅ [462/871] Loaded 1 docs
✅ [463/871] Loaded 1 docs
✅ [464/871] Loaded 1 docs
✅ [465/871] Loaded 1 docs
✅ [466/871] Loaded 1 docs
✅ [467/871] Loaded 1 docs
✅ [468/871] Loaded 1 docs
✅ [469/871] Loaded 1 docs
💾 Saved 454 docs so far at step 470.
✅ [470/871] Loaded 1 docs
✅ [471/871] Loaded 1 docs
✅ [472/871] Loaded 1 docs
✅ [473/871] Loaded 1 docs
✅ [474/871] Loaded 1 docs
✅ [475/871] Loaded 1 docs
✅ [476/871] Loaded 1 docs
✅ [477/871] Loaded 1 docs
✅ [478/871] Loaded 1 docs
✅ [479/871] Loaded 1 docs
💾 Saved 464 docs so far at step 480.
✅ [480/871] Loaded 1 docs
✅ [481/871] Loaded 1 docs
✅ [482/871] Loaded 1 docs
✅ [483/871] Loaded 1 docs
✅ [484/871] Loaded 1 docs
✅ [485/871] Loaded 1 docs


Error fetching or processing https://www.indiacode.nic.in/bitstream/123456789/1520/5/A1963-11.pdf, exception: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


✅ [486/871] Loaded 0 docs
✅ [487/871] Loaded 1 docs
✅ [488/871] Loaded 1 docs
✅ [489/871] Loaded 1 docs
💾 Saved 473 docs so far at step 490.
✅ [490/871] Loaded 1 docs
✅ [491/871] Loaded 1 docs
✅ [492/871] Loaded 1 docs
✅ [493/871] Loaded 1 docs
✅ [494/871] Loaded 1 docs
✅ [495/871] Loaded 1 docs
✅ [496/871] Loaded 1 docs
✅ [497/871] Loaded 1 docs
✅ [498/871] Loaded 1 docs
✅ [499/871] Loaded 1 docs
💾 Saved 483 docs so far at step 500.
✅ [500/871] Loaded 1 docs
✅ [501/871] Loaded 1 docs
✅ [502/871] Loaded 1 docs
✅ [503/871] Loaded 1 docs
✅ [504/871] Loaded 1 docs
✅ [505/871] Loaded 1 docs
✅ [506/871] Loaded 1 docs
✅ [507/871] Loaded 1 docs
✅ [508/871] Loaded 1 docs
✅ [509/871] Loaded 1 docs
💾 Saved 493 docs so far at step 510.
✅ [510/871] Loaded 1 docs
✅ [511/871] Loaded 1 docs
✅ [512/871] Loaded 1 docs
✅ [513/871] Loaded 1 docs
✅ [514/871] Loaded 1 docs
✅ [515/871] Loaded 1 docs
✅ [516/871] Loaded 1 docs
✅ [517/871] Loaded 1 docs
✅ [518/871] Loaded 1 docs
✅ [519/871] Loaded 1 docs
💾 Sav

Error fetching or processing https://www.indiacode.nic.in/bitstream/123456789/1606/3/a1971-59.pdf, exception: HTTPSConnectionPool(host='www.indiacode.nic.in', port=443): Max retries exceeded with url: /bitstream/123456789/1606/3/a1971-59.pdf (Caused by SSLError(SSLError(1, '[SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2578)')))


✅ [563/871] Loaded 0 docs
✅ [564/871] Loaded 1 docs
✅ [565/871] Loaded 1 docs
✅ [566/871] Loaded 1 docs
✅ [567/871] Loaded 1 docs
✅ [568/871] Loaded 1 docs
✅ [569/871] Loaded 1 docs
💾 Saved 552 docs so far at step 570.
✅ [570/871] Loaded 1 docs
✅ [571/871] Loaded 1 docs
✅ [572/871] Loaded 1 docs
✅ [573/871] Loaded 1 docs
✅ [574/871] Loaded 1 docs
✅ [575/871] Loaded 1 docs
✅ [576/871] Loaded 1 docs
✅ [577/871] Loaded 1 docs
✅ [578/871] Loaded 1 docs
✅ [579/871] Loaded 1 docs
💾 Saved 562 docs so far at step 580.
✅ [580/871] Loaded 1 docs
✅ [581/871] Loaded 1 docs
✅ [582/871] Loaded 1 docs
✅ [583/871] Loaded 1 docs
✅ [584/871] Loaded 1 docs
✅ [585/871] Loaded 1 docs
✅ [586/871] Loaded 1 docs
✅ [587/871] Loaded 1 docs
✅ [588/871] Loaded 1 docs
✅ [589/871] Loaded 1 docs
💾 Saved 572 docs so far at step 590.
✅ [590/871] Loaded 1 docs
✅ [591/871] Loaded 1 docs
✅ [592/871] Loaded 1 docs
✅ [593/871] Loaded 1 docs
✅ [594/871] Loaded 1 docs
✅ [595/871] Loaded 1 docs
✅ [596/871] Loaded 1 docs
✅ [59